<a href="https://colab.research.google.com/github/bryaanabraham/Wandering_Guardian/blob/main/ML%20Implementation/ML_Navigation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install gpxpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


In [9]:
import gpxpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
with open('./ONER_2023_FN.gpx') as fh:
  gpx_file = gpxpy.parse(fh)

segment = gpx_file.tracks[0].segments[0]
coords = pd.DataFrame([
    {'lat': p.latitude,
     'lon': p.longitude,
     'ele': p.elevation,
     } for p in segment.points])
coords.head()

,lat,lon,ele
0,50.73446,-2.90010,3.400000
1,50.73446,-2.90010,3.400000
2,50.73425,-2.89969,2.200000
3,50.73431,-2.89961,2.175167
4,50.73427,-2.89954,2.156287


In [11]:
# Deviation parameters
min_deviation_distance = -0.0004
max_deviation_distance = 0.0004
max_deviation_angle = 0

# Generate random deviation distances and angles for each point in the DataFrame
num_points = len(coords)
random_deviation_distances = np.random.uniform(min_deviation_distance, max_deviation_distance, num_points)
random_deviation_angles = np.random.uniform(0, max_deviation_angle, num_points)

# Convert random angles to radians
random_deviation_angles_radians = np.radians(random_deviation_angles)

# Calculate deviation in latitude and longitudet
deviation_distance_lat = random_deviation_distances * np.cos(random_deviation_angles_radians)
deviation_distance_lon = random_deviation_distances * np.sin(random_deviation_angles_radians)

decoy_coords = coords.copy()
decoy_coords['lat'] += deviation_distance_lat
decoy_coords['lon'] += deviation_distance_lon

decoy_coords.head()


,lat,lon,ele
0,50.734373,-2.90010,3.400000
1,50.734146,-2.90010,3.400000
2,50.733867,-2.89969,2.200000
3,50.734167,-2.89961,2.175167
4,50.734052,-2.89954,2.156287


In [15]:
import folium

m = folium.Map()

gps_path = list(zip(coords['lat'], coords['lon']))
folium.PolyLine(locations=gps_path, color='blue', weight=2.5, opacity=0.8).add_to(m)

gps_path = list(zip(decoy_coords['lat'], decoy_coords['lon']))
folium.PolyLine(locations=gps_path, color='red', weight=2.5, opacity=0.8).add_to(m)

max_lat = max(coords['lat'])
min_lat = min(coords['lat'])
min_lon = min(coords['lon'])
max_lon = max(coords['lon'])
m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])

m

In [17]:
from geopy.distance import geodesic

def calculate_deviation(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

deviations = []

# Iterate through every 100th index and calculate the distance between coords and decoy_coords
for i in range(0, min(len(coords), len(decoy_coords)), 100):
    deviation = calculate_deviation((coords.iloc[i]['lat'], coords.iloc[i]['lon']),
                                  (decoy_coords.iloc[i]['lat'], decoy_coords.iloc[i]['lon']))
    deviations.append((i, deviation))

for index, deviation in deviations:
    coord = (decoy_coords.iloc[index]['lat'], decoy_coords.iloc[index]['lon'])
    if deviation <= 0.02:
      colour = 'green'
    elif deviation >= 0.04:
      colour = 'red'
    else:
      colour = 'orange'
    m.add_child(folium.Marker(location=coord,
                              popup=f"Deviation: {deviation:.2f} km",
                              icon=folium.Icon(color=colour)))
m